## Process JUMP SQLite files

1. Load file paths from AWS
2. Append perturbation types from https://github.com/jump-cellpainting/datasets
3. Output file manifest

In [1]:
import boto3
import pathlib
import pandas as pd

In [2]:
# Set constants
jump_id = "cpg0016-jump"
s3_bucket_name = "cellpainting-gallery"
backend_directory = "workspace/backend/"

jump_data_commit = "4b24577c2d3228d92177b807fa53fbbc623da1cb"
jump_github = "https://github.com/jump-cellpainting/datasets/"
perturbation_type_url = f"{jump_github}/raw/{jump_data_commit}/metadata/plate.csv.gz"

output_file = pathlib.Path("metadata", "jump_sqlite_aws_file_locations.tsv")

In [3]:
# Connect to Cell Painting gallery S3 session
# Note: you must have already configured awscli with credentials
session = boto3.Session()
s3 = session.resource("s3")

cell_painting_gallery = s3.Bucket(s3_bucket_name)

In [4]:
# Use client to list source directories
# Sources represent different sites from the JUMP consortium
client = boto3.client("s3")

top_level_jump_result = client.list_objects(
    Bucket=s3_bucket_name,
    Prefix=f"{jump_id}/",
    Delimiter="/"
)

data_collection_sources = []
for source_directory in top_level_jump_result.get("CommonPrefixes"):
    data_collection_sources.append(source_directory.get("Prefix").split("/")[1])
    
data_collection_sources

['source_1',
 'source_10',
 'source_11',
 'source_13',
 'source_2',
 'source_3',
 'source_4',
 'source_5',
 'source_6',
 'source_7',
 'source_8',
 'source_9']

In [5]:
# Obtain batch names for each source
source_batch_dictionary = {}
for source in data_collection_sources:
    prefix_string_per_source = f"{jump_id}/{source}/{backend_directory}"
    
    source_prefix_where_batches_exist_client = client.list_objects(
        Bucket=s3_bucket_name,
        Prefix=prefix_string_per_source,
        Delimiter="/"
    )
    source_batch_dictionary[source] = []
    for batch_directory in source_prefix_where_batches_exist_client.get("CommonPrefixes"):
        source_batch_dictionary[source].append(batch_directory["Prefix"].split("/")[-2])
        
source_batch_dictionary

{'source_1': ['Batch1_20221004',
  'Batch2_20221006',
  'Batch3_20221010',
  'Batch4_20221012',
  'Batch5_20221030',
  'Batch6_20221102'],
 'source_10': ['2021_05_31_U2OS_48_hr_run1',
  '2021_06_01_U2OS_48_hr_run2',
  '2021_06_07_U2OS_48_hr_run3',
  '2021_06_08_U2OS_48_hr_run4',
  '2021_06_14_U2OS_48_hr_run5',
  '2021_06_15_U2OS_48_hr_run6',
  '2021_06_21_U2OS_48_hr_run7',
  '2021_06_22_U2OS_48_hr_run8',
  '2021_06_28_U2OS_48_hr_run9',
  '2021_07_05_U2OS_48_hr_run10',
  '2021_07_07_U2OS_48_hr_run11',
  '2021_08_03_U2OS_48_hr_run12',
  '2021_08_09_U2OS_48_hr_run13',
  '2021_08_12_U2OS_48_hr_run15',
  '2021_08_17_U2OS_48_hr_run16',
  '2021_08_20_U2OS_48_hr_run17',
  '2021_08_23_U2OS_48_hr_run18',
  '2021_08_23_U2OS_48_hr_run19'],
 'source_11': ['Batch1', 'Batch2', 'Batch3', 'Batch4', 'Batch5'],
 'source_13': ['20220914_Run1',
  '20221009_Run2',
  '20221017_Run3',
  '20221024_Run4',
  '20221109_Run5',
  '20221120_Run6'],
 'source_2': ['20210607_Batch_2',
  '20210614_Batch_1',
  '20210621_

In [6]:
# Get plate ids
source_plate_dictionary = {}
source_plate_info = []
for source in source_batch_dictionary:
    source_plate_dictionary[source] = {}
    source_batches = source_batch_dictionary[source]
    
    for batch in source_batches:        
        prefix_string_per_batch = f"{jump_id}/{source}/{backend_directory}{batch}/"

        source_prefix_where_plates_exist_client = client.list_objects(
            Bucket=s3_bucket_name,
            Prefix=prefix_string_per_batch,
            Delimiter="/"
        )
        source_plate_dictionary[source][batch] = []

        for plate_directory in source_prefix_where_plates_exist_client.get("CommonPrefixes"):
            plate = plate_directory["Prefix"].split("/")[-2]
            source_plate_dictionary[source][batch].append(plate)
            
            # Build SQLite file
            sqlite_file = f"s3://{s3_bucket_name}/{prefix_string_per_batch}{plate}/{plate}.sqlite"
            source_plate_info.append([source, batch, plate, sqlite_file])

In [7]:
# Compile JUMP data manifest
jump_df = pd.DataFrame(source_plate_info, columns=["soure", "batch", "plate", "sqlite_file"])

# Append perturbation type to this dataframe
# Load metadata from https://github.com/jump-cellpainting/datasets
perturbation_type_df = pd.read_csv(perturbation_type_url)
perturbation_cols = ["Metadata_Source", "Metadata_Batch", "Metadata_Plate"]

jump_df = (
    jump_df.merge(
        perturbation_type_df,
        left_on=["soure", "batch", "plate"],
        right_on=perturbation_cols,
        how="outer"
    )
    .drop(perturbation_cols, axis="columns")
)

# Output file
jump_df.to_csv(output_file, sep="\t", index=False)

print(jump_df.shape)
jump_df.head()

(2380, 5)


,soure,batch,plate,sqlite_file,Metadata_PlateType
0,source_1,Batch1_20221004,UL000109,s3://cellpainting-gallery/cpg0016-jump/source_...,COMPOUND_EMPTY
1,source_1,Batch1_20221004,UL001641,s3://cellpainting-gallery/cpg0016-jump/source_...,COMPOUND
2,source_1,Batch1_20221004,UL001643,s3://cellpainting-gallery/cpg0016-jump/source_...,COMPOUND
3,source_1,Batch1_20221004,UL001645,s3://cellpainting-gallery/cpg0016-jump/source_...,COMPOUND
4,source_1,Batch1_20221004,UL001651,s3://cellpainting-gallery/cpg0016-jump/source_...,COMPOUND


In [8]:
jump_df.Metadata_PlateType.value_counts()

Metadata_PlateType
COMPOUND          1729
ORF                251
CRISPR             148
TARGET2            141
DMSO                77
COMPOUND_EMPTY      20
POSCON8              8
TARGET1              4
Name: count, dtype: int64